In [147]:
import collections
import copy
from tqdm import tqdm
import itertools
from multiprocessing import Pool, Process
import numpy as np
import statistics
from IPython.display import clear_output

In [102]:
knowledgebase = []
with open("4Bdata.txt", "r") as f:
    knowledgebase = f.read().split("\n")[:-1]
    
print(knowledgebase[-1])

proszęozabraniegłosuprezesanajwyższejizbykontrolipanamirosławasekułę


In [159]:
def runbpe(sentence):
    wfreq = collections.defaultdict(lambda: 0)
    if len(sentence) == 1:
        return wfreq
    
    for wl, wp in zip(sentence[:-1], sentence[1:]):
        wfreq[wl + "|" + wp] += 1
    return wfreq

def replace_in_set(sentences, replacemap):
    targets = set(replacemap.keys())
    nset = []
    for line in sentences:
        if len(line) == 1:
            nset.append(copy.deepcopy(line))
        else:
            nline = []
            skip = False
            for wl, wp in zip(line[:-1], line[1:]):     
                ntok = wl + "|" + wp
                if ntok in targets:
                    skip = True
                    nline.append(replacemap[ntok])
                else:
                    if skip == True:
                        skip = False
                    else:
                        nline.append(wl)
            if skip == False:
                nline.append(line[-1])
            nset.append(copy.deepcopy(nline))
    return nset

replace_in_set(["aaaaaaabbbu"], {"b|u":"[k]"})

[['a', 'a', 'a', 'a', 'a', 'a', 'a', 'b', 'b', '[k]']]

In [175]:
def rungrpbpe(sentences):
    dicts = []
    for sentence in sentences:
        dicts.append(runbpe(sentence))
        
    wfreq = collections.defaultdict(lambda: 0)
    for d in dicts:
        for dk in d.keys():
            wfreq[dk] += d[dk]

    cdump = list(sorted([(wfreq[w], w) for w in wfreq.keys()], reverse=True))
    possible = []
    used = set()
    okfreq = 0
    consoom = True
    for c in cdump:
        cl, cp = c[1].split("|")
        if consoom:
            if cl in used or cp in used:
                consoom = False
            else:
                possible.append(copy.deepcopy(c))
                used |= set([cl, cp])
                okfreq = c[0]
        else:
            if c[0] < okfreq:
                break
            if cl in used or cp in used:
                pass
            else:
                possible.append(copy.deepcopy(c))
                used |= set([cl, cp])
                okfreq = c[0]            
    return (possible, sum(u[0] for u in possible))

In [176]:
#runbpe(list(knowledgebase[0]))
splitset = [list(kb) for kb in knowledgebase]

tokid = 1
for i in range(5):
    splittrgt, splitcnt = rungrpbpe(splitset)
    rplmap = {}
    for splt in splittrgt:
        rplmap[splt[1]] = f"[tok{tokid}]"
        tokid += 1
    
    print(splitcnt, rplmap)
    splitset = replace_in_set(splitset, rplmap)

80958 {'i|e': '[tok1]'}
131821 {'o|w': '[tok2]', 'a|n': '[tok3]', 's|t': '[tok4]'}
103402 {'p|r': '[tok5]', 'c|h': '[tok6]', 'z|e': '[tok7]'}
63712 {'p|o': '[tok8]', 'n|a': '[tok9]'}
27089 {'n|i': '[tok10]'}


In [184]:
splitset = [list(kb) for kb in knowledgebase]
knowledge = collections.defaultdict(lambda: 0)
rendered = collections.defaultdict(lambda: "")
for letter in set(list(itertools.chain(*splitset))):
    knowledge[letter] = 1
    rendered[letter] = letter
    
def calculate_wordlen(sentences, knowledge):
    wlen = 0
    wdcnt = 0
    for line in sentences:
        for word in line:
            wdcnt += 1
            wlen += knowledge[word]
    return wlen/wdcnt

tokid = 0
decomposition = []
for i in range(5000000000):
    splittrgt, splitcnt = rungrpbpe(splitset)
    rplmap = {}
    for splt in splittrgt:
        rplmap[splt[1]] = f"[tok{tokid}]"
        tokid += 1
    
    print(splitcnt, rplmap)
    splitset = replace_in_set(splitset, rplmap)
    
    for replacement_key in rplmap.keys():
        print(replacement_key, rplmap[replacement_key])
        knowledge[rplmap[replacement_key]] = knowledge[replacement_key.split("|")[0]] + knowledge[replacement_key.split("|")[1]] 
        rendered[rplmap[replacement_key]] = rendered[replacement_key.split("|")[0]] + rendered[replacement_key.split("|")[1]]
        decomposition.append(tuple([rplmap[replacement_key], replacement_key.split("|")[0], replacement_key.split("|")[1]]))
    
    
    wlen = calculate_wordlen(splitset, knowledge)
    print(f"avg word length = {wlen}")
    print(splitset[0])
    clear_output(wait=True)
    if wlen > 6.2:
        break
    

417 {'z|[tok5345]': '[tok45904]', 'w|[tok8195]': '[tok45905]', 'u|[tok41443]': '[tok45906]', 's|[tok189]': '[tok45907]', 'p|[tok14345]': '[tok45908]', 'o|[tok4360]': '[tok45909]', 'l|[tok1641]': '[tok45910]', 'k|[tok196]': '[tok45911]', 'i|[tok596]': '[tok45912]', 'd|[tok132]': '[tok45913]', 'c|[tok31827]': '[tok45914]', 'a|[tok24313]': '[tok45915]', '[tok9]|[tok12229]': '[tok45916]', '[tok98]|[tok2706]': '[tok45917]', '[tok97]|[tok12]': '[tok45918]', '[tok92]|[tok3281]': '[tok45919]', '[tok9187]|[tok45765]': '[tok45920]', '[tok8]|[tok7330]': '[tok45921]', '[tok87]|[tok2209]': '[tok45922]', '[tok8647]|m': '[tok45923]', '[tok8505]|t': '[tok45924]', '[tok83]|[tok84]': '[tok45925]', '[tok7]|[tok6461]': '[tok45926]', '[tok72]|[tok11071]': '[tok45927]', '[tok7293]|ł': '[tok45928]', '[tok70]|[tok5824]': '[tok45929]', '[tok6]|[tok36060]': '[tok45930]', '[tok66]|[tok1066]': '[tok45931]', '[tok64]|[tok400]': '[tok45932]', '[tok6462]|e': '[tok45933]', '[tok63]|[tok91]': '[tok45934]', '[tok61]|[t

In [131]:
for line in splitset[:5]:
    print(line)

['[tok10809]', '[tok374]', '[tok16]', '[tok288]', '[tok45]', '[tok35]', '[tok20191]', '[tok13191]', '[tok1643]', '[tok25249]']
['[tok300]', '[tok281]', '[tok37018]', '[tok1260]', '[tok27066]', '[tok977]', '[tok24526]', 's', '[tok13167]', '[tok4431]', '[tok25650]']
['[tok30750]', '[tok1205]', '[tok886]', '[tok23862]', '[tok3604]', '[tok1021]', 'i', '[tok3434]', '[tok1068]', '[tok1328]', '[tok391]', '[tok35494]']
['[tok1264]', '[tok336]', '[tok13673]', '[tok63]', '[tok850]', '[tok6743]', '[tok9903]', '[tok28395]', '[tok19464]', '[tok7271]', '[tok2038]', '[tok3201]', '[tok4732]', '[tok162]', '[tok8682]', '[tok1317]', '[tok10333]', '[tok26201]']
['[tok1342]', 'w', '[tok2408]', '[tok3737]', '[tok734]', '[tok11442]', '[tok2]', '[tok3451]', '[tok238]', '[tok677]', '[tok15138]', '[tok13114]', '[tok286]', '[tok15148]']


In [134]:
for rkey in list(rendered.keys())[:64]:
    print(rkey, rendered[rkey])

k k
ć ć
w w
ś ś
u u
d d
m m
n n
t t
ł ł
r r
e e
o o
ó ó
p p
j j
s s
f f
c c
b b
i i
ę ę
z z
ą ą
h h
ż ż
g g
a a
ź ź
l l
y y
[tok0] ie
[tok1] ow
[tok2] an
[tok3] st
[tok4] pr
[tok5] ch
[tok6] ze
[tok7] po
[tok8] na
[tok9] ni
[tok10] cz
[tok11] nie
[tok12] za
[tok13] je
[tok14] od
[tok15] ia
[tok16] ra
[tok17] zy
[tok18] ko
[tok19] te
[tok20] aw
[tok21] ne
[tok22] mi
[tok23] ar
[tok24] li
[tok25] go
[tok26] ny
[tok27] ów
[tok28] dz
[tok29] do
[tok30] ro
[tok31] ty
[tok32] le


In [178]:
splitset = [list(kb) for kb in knowledgebase][0]
print(splitset)
print("".join(splitset))
print(decomposition[0])
for dcm in decomposition:
    splitset = replace_in_set([splitset], {dcm[1] + "|" + dcm[2] : dcm[0]})[0]
    #print(dcm, "".join(splitset))

print(splitset)
for e in splitset:
    print(rendered[e])

['s', 'y', 's', 't', 'e', 'm', 'e', 'u', 'r', 'o', 'r', 'a', 't', 'y', 'c', 'h', 'c', 'e', 's', 'z', 'k', 'u', 'p', 'o', 'w', 'a', 'ć', 'w', 'i', 'ę', 'c', 'e', 'j', 'n', 'i', 'ż', 'g', 'd', 'z', 'i', 'e', 'i', 'n', 'd', 'z', 'i', 'e', 'j']
systemeuroratychceszkupowaćwięcejniżgdzieindziej
('[tok0]', 'i', 'e')
['[tok10996]', '[tok374]', '[tok16]', '[tok288]', '[tok45]', '[tok35]', '[tok19904]', '[tok12841]', '[tok1643]', '[tok26428]']
systeme
uro
ra
tych
ce
sz
kupować
więcejniż
gdzie
indziej


In [185]:
def finalize_splitting(word, decomposition):
    split = list(word)
    for dcm in decomposition:
        split = replace_in_set([split], {dcm[1] + "|" + dcm[2] : dcm[0]})[0]
    return split

print(finalize_splitting("doświadczenia", decomposition))
print(finalize_splitting("mieniłkonstyt", decomposition))

def untokenize(tok):
    try:
        return int(tok[4:-1])
    except Exception as e:
        return 100000000
    
print([untokenize(t) for t in finalize_splitting("mieniłkonstyt", decomposition)])

['[tok5748]']
['[tok1315]', 'ł', '[tok128]', '[tok167]', 't']
[1315, 100000000, 128, 167, 100000000]


In [188]:
def procbatch(batch):
    scoring = []
    for t in batch:
        tp, tw = t.split()
        sp = finalize_splitting(tp, decomposition)
        sw = finalize_splitting(tw, decomposition)
        
        if len(sp) == len(sw):
            pscore = sum([untokenize(t) for t in sp])
            wscore = sum([untokenize(t) for t in sw])
            if pscore < wscore:
                scoring.append(1.0)
            elif pscore == wscore:
                scoring.append(0.5)
            else:
                scoring.append(0.0)
        elif len(sp) < len(sw):
            scoring.append(1.0)
        else:
            scoring.append(0.0)
    return scoring

with open("4Btest.txt", "r") as f:
    scoring = []
    tests = f.read().split("\n")[:-1]

    batched = np.array_split(tests, 32)
    
    with Pool(32) as p:
        results = p.map(procbatch, batched)
    
    print(statistics.mean(list(itertools.chain(*results))))
    

0.77442
